In [1]:
import xlwt 
from xlwt import Workbook 
import math
import re
import sys
import csv
from googletrans import Translator

sentiment={}
text=[]
en = 'AFINN-111.txt'
a=lambda w, s: (w, int(s)), [ws.strip().split('\t') for ws in open(en) ]

for ws in open('positive_words_pt.txt' , encoding='utf8'):
    sentiment[ws.replace('\n','')]=1
for ws in open('negative_words_pt.txt' , encoding='utf8'):
    sentiment[ws.replace('\n','')]=-1
for ws in open('positive_words_hi.txt' , encoding='utf8'):
    sentiment[ws.replace('\n','')]=1
for ws in open('negative_words_hi.txt' , encoding='utf8'):
    sentiment[ws.replace('\n','')]=-1
for val in a[1]:
    sentiment[val[0]]=int(val[1])

In [2]:
def clean_tweet(tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

def get_tweet_sentiment(tweet,lang): 

        # set sentiment 
        if lang != 'en':
            score=0
            for val in tweet.split():
                if sentiment.get(val):
                    score+=sentiment.get(val)
            if score > 0: 
                return 'positive'
            elif score == 0: 
                return 'neutral'
            else: 
                return 'negative'

        analysis = TextBlob((tweet))
        analysis = TextBlob(clean_tweet(tweet))
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'


def round_to_hour(dt):
    dt_start_of_hour = dt.replace(minute=0, second=0, microsecond=0)
    dt_half_hour = dt.replace(minute=30, second=0, microsecond=0)

    if dt >= dt_half_hour:
        # round up
        dt = dt_start_of_hour + datetime.timedelta(hours=1)
    else:
        # round down
        dt = dt_start_of_hour
    return dt


def convertToObj(tweet,country,topic):
    text.append(tweet.full_text)
    obj={}
    obj["poi_name"] = tweet.user.screen_name
    obj["name"] = tweet.user.name
    obj["location"] = tweet.user.location

    obj["poi_id"] = tweet.user.id
    obj["verified"] = tweet.user.verified
    obj["country"] = country
    obj["replied_to_tweet_id"] = tweet.in_reply_to_status_id_str
    obj["replied_to_user_id"] = tweet.in_reply_to_user_id_str
    obj["tweet_lang"] = tweet.lang
    obj["hashtags"] = tweet.entities["hashtags"]
    obj["mentions"] = tweet.entities["user_mentions"]
    obj["tweet_urls"] = tweet.entities["urls"]
    obj["tweet_emoticons" ] = ''.join(c for c in tweet.full_text if c in emoji.UNICODE_EMOJI)
    obj["tweet_date"] = str(round_to_hour(tweet.created_at))
    obj["id"]  = tweet.id
    obj["link"] = "twitter.com/user/status/"+str(tweet.id)
    obj["geo"]=tweet.geo
    if tweet.place is not None:
        obj["city"]=tweet._json['place']['full_name']
        obj["coordinate"]=tweet._json['place']['bounding_box']
        
    obj['like_count']=tweet.favorite_count
    obj['retweet_count']=tweet.retweet_count

    if tweet.in_reply_to_user_id:
        obj["reply_text"] = tweet.full_text
    obj["tweet_text"] = tweet.full_text
    
    obj["topic"]=topic
    obj["sentiment"]=get_tweet_sentiment(tweet.full_text,tweet.lang)
    return obj



In [3]:
#Initialize widgets
import ipywidgets as widgets
from tweepy.error import TweepError

POI = widgets.Dropdown(
    options=['narendramodi','thekiranbedi','AmitShah','ShashiTharoor','nitin_gadkari','ptshrikant'
             ,'AOC','BarackObama','SarahPalinUSA','megynkelly','tedcruz','realDonaldTrump','ewarren','SenSanders'
             ,'rafinhabastos','dilmabr','jairbolsonaro','MarinaSilva','joseserra_'],
    value='narendramodi',
    description='POI:',
    disabled=False,
)

display(POI)
Country = widgets.Dropdown(
    options=['India', 'USA', 'Brazil'],
    value='India',
    description='Country:',
    disabled=False,
)
display(Country)

tweet_count = widgets.IntSlider(
    description='Tweet Count:',
    min=0,
    max=1000,
    value=250,
    step=10)
display(tweet_count)

Dropdown(description='POI:', options=('narendramodi', 'thekiranbedi', 'AmitShah', 'ShashiTharoor', 'nitin_gadk…

Dropdown(description='Country:', options=('India', 'USA', 'Brazil'), value='India')

IntSlider(value=250, description='Tweet Count:', max=1000, step=10)

In [7]:
#Import parameters and set variables
import tweepy
import json
from datetime import date, time, datetime
import datetime
import sys
import time
from textblob import TextBlob 
import emoji
import re
from googletrans import Translator


temp=""


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
count=0
retweet_count =0


tweet_dict = {}
counter =0
target = POI.value
country = Country.value
tweet_cnt=tweet_count.value



In [8]:
topic="Disaster"
query=["disaster","flood","hurricane","earthquake","wildfire","bushfire","fire","deforestation","tsunami","tornado","draught","global warming","ice cap melt","volcano eruption","storm","snowstorm","heatstroke","smog","pollution","endangered species","extinction","accident","flood","आपदा", "बाढ़", "तूफान", "भूकंप", "जंगल की आग", "झाड़-फूंक", "आग", "वनों की कटाई", "सुनामी", "बवंडर", "मसौदा", "ग्लोबल वार्मिंग", "आइस कैप पिघलता है","ज्वालामुखी विस्फोट","तूफान","स्नोस्टॉर्म"," ीटस्ट्रोक","स्मॉग","प्रदूषण","लुप्तप्राय प्रजातियां","विलुप्त होने","दुर्घटना"," ाढ़","desastre", "inundação", "furacão", "terremoto", "incêndio", "incêndio florestal", "incêndio", "desmatamento", "tsunami", "tornado", "calado", "aquecimento global", "derretimento da calota de gelo","erupção vulcânica","tempestade","tempestade de neve","insolação","poluição atmosférica","poluição","poluição","espécies ameaçadas de extinção","extinção","acidente","inundação"]

In [12]:
topic="Religion"
query=["Ram","Shiva","Hindu","Muslim","prayer","miracle","pastor","Jesus","Allah","Krishna","God","festival","celebration","fast","Diwali","Christmas","religion","faith","belief","worship","followers","devotee","devotion","temple","church","mosque","pilgrimage","eid","ramzan","hajj","new year","Ram", "Shiva", "Hindu", "Muçulmano", "oração", "milagre", "pastor", "Jesus", "Deus", "Krishna", "Deus", "festival", "celebração","rápido","Diwali","Natal","religião","fé","crença","adoração","seguidores","devoto","devoção","templo","igreja", "mesquita", "peregrinação", "eid", "ramzan", "hajj", "ano novo","राम", "शिव", "हिन्दू", "मुस्लिम", "प्रार्थना", "चमत्कार", "पादरी", "यीशु", "अल्लाह", "कृष्णा", "भगवान", "त्योहार", "उत्सव","तेज"," ीवाली","क्रिसमस","धर्म"," िश्वास"," िश्वास","पूजा","अनुयायियों","भक्त","भक्ति","मंदिर","चर्च", "मस्जिद", "तीर्थयात्रा", "ईद", "रमजान", "हज", "नया साल"]

In [16]:
topic="Politics"
query=["politics","politician","leader","party","president","prime minister","parliament","election","bill","vote","voters","white house","amendment","law","section 377","right","immigration","healthcare","government","opposition","campaign","tax","GST","senate","senator","राजनीति", "राजनीतिज्ञ", "नेता", "पार्टी", "राष्ट्रपति", "प्रधान मंत्री", "संसद", "चुनाव", "बिल", "वोट", "मतदाता", "सफेद घर", "संशोधन", "कानून", "धारा 377", "सही", "आव्रजन", "स्वास्थ्य सेवा", "सरकार", "विपक्ष", "अभियान", "कर", "जीएसटी", "सीनेट", " ीनेटर","política", "político", "líder", "partido", "presidente", "primeiro ministro", "parlamento", "eleição", "projeto de lei", "voto", "eleitor", "casa branca", "emenda", "lei", "seção 377", "direita", "imigração", "assistência médica", "governo", "oposição", "campanha", "imposto", "GST", "senado", "senador"] 

In [20]:
topic="Crime"
query=["forgery","crime","DUI","catcall","kidnap","trafficking","drug","harrassment","arrest","robbery","kill","fight","thief","jail","punish","murder","prison","rape","mugging","attack","hijack","scared","psycho","gun","accident","hacking","threaten","blackmail","aggression","FBI","falsificação", "crime", "DUI", "catcall", "seqüestro", "tráfico", "droga", "assédio", "prisão", "assalto", "assalto", "matar", "brigar" ,"ladrão","prisão","punir","assassinato","prisão","estupro","assalto","ataque","seqüestro","medo","psicopata","arma","acidente", "hackers", "ameaçar", "chantagem", "agressão", "FBI","जालसाजी", "अपराध", "DUI", "सन्नाटा", "अपहरण", "अवैध व्यापार", "दवा", "उत्पीड़न", "गिरफ्तार", "डकैती", "मार", "लड़ाई", "चोर","जेल","सज़ा","हत्या","जेल","बलात्कार","लूट","हमले","अपहरण","डर","पागल","बंदूक","दुर्घटना", "हैकिंग", "धमकी", "ब्लैकमेल", "आक्रामकता", "एफबीआई"]


In [25]:
topic="Entertainment"
query=["entertainment","celebrate","movie","sports","games","actor","actress","singer","band","fun","anime","premiere","release","album","pop","rock","concert","celebrity","performance","मनोरंजन", "जश्न मनाने", "फिल्म", "खेल", "खेल", "अभिनेता", "अभिनेत्री", "गायक", "बैंड", "मज़ा", "एनिमी", "प्रीमियर", "रिहाई","एल्बम","पॉप","रॉक","संगीत कार्यक्रम","सेलिब्रिटी","प्रदर्शन","lançamento de","entretenimento", "comemorar", "filme", "esportes", "jogos", "ator", "atriz", "antor", "banda", "diversão", "anime", "estréia","álbum","pop","rock","concerto","celebridade","performance"]

In [27]:
topic="General"


In [30]:
#Extract main tweets
counter=0
count=0
tweet_cnt=2000
for tweet in tweepy.Cursor(api.user_timeline, id="@" + target, q=query, tweet_mode="extended",result_type="recent").items():
    counter=counter+1 
    retweet_count = retweet_count+1

    if  tweet._json["id"] not in tweet_dict and not hasattr(tweet, 'retweeted_status') and not tweet.in_reply_to_status_id:# and any(word in tweet.full_text for word in query):
#        #not hasattr(tweet, 'retweeted_status') and not tweet.in_reply_to_status_id and
        count=count + 1
        print(count)
        with open('P4_'+topic+'_'+target+'tweet.json', 'a+',encoding='utf8' ) as f:
            obj = convertToObj(tweet,country,topic)
            json.dump(obj, f, ensure_ascii=False)
        tweet_dict[tweet._json["id"]]=0
            
    if counter%100==0:
        time.sleep(10)
    
    if len(tweet_dict) >tweet_cnt:
        break
    


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
##########################################################################################################################
################################################Topic Modelling###########################################################
##########################################################################################################################

In [ ]:
import re
import pandas as pd

tweet=pd.DataFrame(text,columns=['text'])

In [ ]:

# Remove punctuation
tweet['text_processed'] = tweet['text'].map(lambda x: re.sub('[,\.!?]', '', x))
tweet['text_processed'] = tweet['text'].map(lambda x: re.sub('http', '', x))

# Convert the titles to lowercase
tweet['text_processed'] = tweet['text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
tweet['text_processed'].head()

In [ ]:
import nltk
from nltk.corpus import stopwords

stop = stopwords.words("hindi.txt")
stop += stopwords.words("english")
stop += stopwords.words("portuguese")
print(stop)



In [ ]:
words={}
f=[]
for t in tweet['text_processed']:
    for word in t.split():
        if word not in stop:
            f.append(word)
s = {{k, words[k]} for k in sorted(words, key=words.get, reverse=True)}
print(words)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]


In [ ]:
##########################################################################################################################
###################################################Replies################################################################
##########################################################################################################################

print(len(tweet_dict))
print(tweet_cnt)

In [31]:
import pandas as pd
count_map={}
id_list=[]
replies_list=[]
for id in tweet_dict:
    count_map[id] = 0

tweet_id = pd.DataFrame.from_dict(tweet_dict,orient="index")
tweet_id['index']=tweet_id.index
id_list=tweet_id['index']

In [32]:
print(len(id_list))

2001


In [33]:
def extract_replies(poi,id_list,replies_lis,replies_count_map,max_id):
    try:
        poi_replies_file = 'P4_'+topic+'_'+poi + '_replies.json'
        i =0;
        since_id = min(id_list)
        # max_id = None
        # temp_id = None
        total_tweets = 0
        last_max_id = max_id
        extraction_finished = 0;
        while (1):
            print("Fetching tweets for since id :"+str(since_id)+" max id is :"+str(max_id))
            if max_id == 0:
                cur = tweepy.Cursor(api.search, q='to:'+poi, since_id=since_id, count=1000,tweet_mode='extended').items(1000)
            else:
                cur = tweepy.Cursor(api.search, q='to:'+poi, since_id=since_id, max_id=max_id,count=1000,tweet_mode='extended').items(1000)
            for tweet in cur:
                max_id = tweet.id
                if tweet.in_reply_to_status_id in id_list:
                    total_tweets = total_tweets+1
                    if replies_count_map[tweet.in_reply_to_status_id] <= 20:
                        replies_lis.append(tweet)
                        replies_count_map[tweet.in_reply_to_status_id] = replies_count_map[tweet.in_reply_to_status_id] + 1
                        print("Found reply for tweet :"+str(tweet.in_reply_to_status_id) +" ,reply with id : "+str(tweet.id)+" ,count is" + str(replies_count_map[tweet.in_reply_to_status_id]))
                        with open(poi_replies_file, 'a+', encoding="utf8") as f:
                            obj = convertToObj(tweet,country,topic)
                            obj["poi_name"] = poi
                            obj["poi_id"] = obj["replied_to_user_id"]
                            

                            json.dump(obj, f, ensure_ascii=False)
            if last_max_id == max_id or all(v >= 20 for v in replies_count_map.values()):
                break

    except TweepError:
        time.sleep(10)
        extract_replies(poi,id_list,replies_lis,replies_count_map,max_id)

In [34]:
############################################################################################################################

In [35]:
extract_replies(target,id_list,replies_list,count_map,0)

Fetching tweets for since id :553258853731958784 max id is :0
Found reply for tweet :1200562417664741382 ,reply with id : 1201216383440293888 ,count is1
Found reply for tweet :1200562417664741382 ,reply with id : 1201210407710527488 ,count is2
Found reply for tweet :1200562417664741382 ,reply with id : 1201198840772202496 ,count is3
Found reply for tweet :1200562417664741382 ,reply with id : 1201172032282599425 ,count is4
Found reply for tweet :1200101569271865345 ,reply with id : 1201169568141627393 ,count is1
Found reply for tweet :1200562417664741382 ,reply with id : 1201169491679420419 ,count is5
Found reply for tweet :1200562417664741382 ,reply with id : 1201168662360670208 ,count is6
Found reply for tweet :1200562417664741382 ,reply with id : 1201168028446146561 ,count is7
Found reply for tweet :1200562417664741382 ,reply with id : 1201164060919828482 ,count is8
Found reply for tweet :1200562417664741382 ,reply with id : 1201161248043352064 ,count is9
Found reply for tweet :12005

KeyboardInterrupt: 

In [ ]:
import tweepy
import json
from datetime import date, time, datetime
import datetime
import sys
import time
from textblob import TextBlob 
import emoji
import re

In [ ]:
analysis = TextBlob(("De reportagem do UOL\nPROCESSO CONTRA CALUNIADOR\nAutor de postagem que fez"))
analysis = analysis.translate(to="en")